# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = "../weather_df.csv" 
cities_df = pd.read_csv(data)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bonfim,3.08,-59.95,75.79,87,67,3.15,BR,1594955805
1,jamestown,42.10,-79.24,73.00,94,90,4.70,US,1594955189
2,khatanga,71.97,102.50,46.08,75,100,19.19,RU,1594955099
3,mahebourg,-20.41,57.70,69.80,77,40,6.93,MU,1594955324
4,nemuro,43.32,145.57,69.80,68,20,9.17,JP,1594955326


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure 
gmaps.configure(api_key=g_key)

#set values for gmaps
city = cities_df[['Lat', 'Lng']].astype(float)
humidity = cities_df['Humidity'].astype(float)


# add heatmap
fig = gmaps.figure(map_type='SATELLITE')



heat_layer = gmaps.heatmap_layer(city, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Create a perfect weather dataframe
perf_weather = cities_df.loc[(cities_df['Max Temp'] >= 70) & (cities_df['Max Temp'] <= 80) & 
                             (cities_df['Wind Speed'] <= 20) & (cities_df['Humidity'] <= 30)]
#Drop N/As
perf_weather = perf_weather.dropna()

perf_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,mount isa,-20.73,139.50,73.40,16,93,8.05,AU,1594955333
175,aswan,24.09,32.91,79.66,25,0,8.25,EG,1594955812
384,diamantino,-14.41,-56.45,71.31,30,0,5.23,BR,1594955823
452,jinchang,38.50,102.17,79.70,28,30,5.06,CN,1594955826


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perf_weather
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
76,mount isa,-20.73,139.50,73.40,16,93,8.05,AU,1594955333,
175,aswan,24.09,32.91,79.66,25,0,8.25,EG,1594955812,
384,diamantino,-14.41,-56.45,71.31,30,0,5.23,BR,1594955823,
452,jinchang,38.50,102.17,79.70,28,30,5.06,CN,1594955826,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up a parameters dictionary
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}
# Use iterrows to iterate through pandas dataframe
    
for index, row in hotel_df.iterrows():
    # Find the lat, lng, and city name from each row in hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # Add location to params dict
    params["location"] = f"{lat}, {lng}"
    print(f"Retrieving Results for Index {index}: {city}.")
    
    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response["results"]
    
   # Find the closest hotel to each city and store the name back in the hotel_df
    try:
        print("Closest hotel to " + city + " is " + results[0]["name"])
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    # Handle exceptions in case there is a KeyError or IndexError...
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display the hotel_df
hotel_df

Retrieving Results for Index 76: mount isa.
Closest hotel to mount isa is Isa Hotel
------------
Retrieving Results for Index 175: aswan.
Closest hotel to aswan is Sofitel Legend Old Cataract Aswan
------------
Retrieving Results for Index 384: diamantino.
Closest hotel to diamantino is Hotel Villa Dos parecis
------------
Retrieving Results for Index 452: jinchang.
Closest hotel to jinchang is Jinchuan Hotel
------------


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
76,mount isa,-20.73,139.50,73.40,16,93,8.05,AU,1594955333,Isa Hotel
175,aswan,24.09,32.91,79.66,25,0,8.25,EG,1594955812,Sofitel Legend Old Cataract Aswan
384,diamantino,-14.41,-56.45,71.31,30,0,5.23,BR,1594955823,Hotel Villa Dos parecis
452,jinchang,38.50,102.17,79.70,28,30,5.06,CN,1594955826,Jinchuan Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
hotel_locations = hotel_df[["Lat", "Lng"]]
hotel_locations

# Display figure
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))